In [ ]:
import os
import tabulate
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from rmgpy.tools.uncertainty import Uncertainty, process_local_results
from rmgpy.tools.canteramodel import get_rmg_species_from_user_species
from rmgpy.species import Species
from rmgpy.tools.plot import parse_csv_data


# these are for showing the flux diagram
from IPython.display import Image


In [ ]:
# Load the model
# Must use annotated chemkin file

model_dir = '/home/moon/rmg/rmg_tools/uncertainty/nheptane'


chemkin_file = os.path.join(model_dir, 'chem_annotated.inp')
dict_file = os.path.join(model_dir, 'species_dictionary.txt')

# Initialize the Uncertainty class instance and load the model
uncertainty = Uncertainty(output_directory=os.path.join(model_dir, 'chemkin'))
uncertainty.load_model(chemkin_file, dict_file)


In [ ]:
# optional - generate flux diagram:

# from rmgpy.tools import fluxdiagram

# input_file = os.path.join(model_dir, 'input.py')
# output_path = os.path.join(model_dir, 'flux_diagram')
# fluxdiagram.create_flux_diagram(input_file, chemkin_file, dict_file, save_path=output_path)

Ethane pyrolysis produces ethylene (C2H4)

In [ ]:
# Map the species to the objects within the Uncertainty class
nheptane = Species().from_smiles('CCCCCCC')
CO2 = Species().from_smiles('O=C=O')
O2 = Species().from_smiles('[O][O]')
OH_rad = Species().from_smiles('[OH]')
Ne = Species().from_smiles('[Ne]')
mapping = get_rmg_species_from_user_species([nheptane, CO2, O2, OH_rad, Ne], uncertainty.species_list)


# TODO get model with He instead of Ne
# equivalence ratio phi, 11 O2 per n-heptane in stoichiometric feed
x_nheptane = 0.005
phi = 1.0
x_O2 = 11.0 * x_nheptane / phi
x_Ne = 1.0 - x_nheptane - x_O2
# print(x_nheptane)
# print(x_O2)
# print(x_Ne)

# Reaction conditions to match at least some Zhang(2016) experimental data
initial_mole_fractions = {
    mapping[nheptane]: x_nheptane,
    mapping[O2]: x_O2,
    mapping[Ne]: x_Ne,
}
T = (1000, 'K')
P_torr = 800.0
P_atm = P_torr / 760.0
P_Pa = P_atm * 101325.0
P_bar = P_Pa / 100000.0

P = (P_bar, 'bar')
termination_time = (0.5, 'ms')
sensitive_species = [mapping[nheptane], mapping[CO2], mapping[O2], mapping[OH_rad]]


In [ ]:
# Perform the sensitivity analysis
uncertainty.sensitivity_analysis(initial_mole_fractions, sensitive_species, T, P, termination_time, number=5, fileformat='.png')

In [ ]:
# NOTE: You must load the database with the same settings which were used to generate the model.
#       This includes any thermo or kinetics libraries which were used.
uncertainty.load_database(
    thermo_libraries=[
        'BurkeH2O2',
        'CurranPentane',
        'FFCM1(-)',
        'primaryThermoLibrary',
        'thermo_DFT_CCSDTF12_BAC',
        'DFT_QCI_thermo',
        'CBS_QB3_1dHR'
    ],
    kinetics_families='default',
    reaction_libraries=[
        'CurranPentane',
        'FFCM1(-)',
        'combustion_core/version5'
    ]
)

In [ ]:
uncertainty.extract_sources_from_model()
uncertainty.assign_parameter_uncertainties()

In [ ]:
m_gav = 0
m_lib = 0
un_gav = 0
un_lib = 0


for i, src_dict in enumerate(uncertainty.species_sources_dict):
    src_dict = uncertainty.species_sources_dict[src_dict]
    if 'GAV' in src_dict.keys():
        m_gav += 1
        un_gav += uncertainty.thermo_input_uncertainties[i]
    elif 'Library' in src_dict.keys():
        m_lib += 1
        un_lib += uncertainty.thermo_input_uncertainties[i]
    else:
        print(src_dict.keys())
print(m_gav, un_gav)
print(m_lib, un_lib)


In [ ]:
def mm(pct):
    return int(np.round(pct * (m_gav + m_lib) / 100.0))
%matplotlib inline
plt.figure(figsize=(6.5, 4), dpi=80)
plt.clf()
plt.pie([m_gav, m_lib], startangle=90, autopct=mm)
ax = plt.gca()
ax.legend(['Group Additivity', 'Library'], loc='right', bbox_to_anchor=(1.5, 0.8))
plt.title('Species Sources')
plt.savefig('species_sources.png')
plt.show()


In [ ]:
%matplotlib inline
plt.clf()
plt.pie([un_gav, un_lib], startangle=90, autopct='%1.1f%%')
ax = plt.gca()
ax.legend(['Group Additivity', 'Library'], loc='right', bbox_to_anchor=(1.8, 0.8))
plt.title('Model Uncertainty of Species')
plt.show()

In [ ]:
n_rate_rules = 0
n_pdep = 0
n_lib = 0
n_training = 0
uncertainty_pdep = 0
uncertainty_rate_rules = 0
uncertainty_lib = 0
uncertainty_train = 0

for i, src_dict in enumerate(uncertainty.reaction_sources_dict):
    src_dict = uncertainty.reaction_sources_dict[src_dict]
    if 'Rate Rules' in src_dict.keys():
        n_rate_rules += 1
        uncertainty_rate_rules += uncertainty.kinetic_input_uncertainties[i]
    elif 'Library' in src_dict.keys():
        n_lib += 1
        uncertainty_lib += uncertainty.kinetic_input_uncertainties[i]
    elif 'PDep' in src_dict.keys():
        n_pdep += 1
        uncertainty_pdep += uncertainty.kinetic_input_uncertainties[i]
    elif 'Training' in src_dict.keys():
        n_training += 1
        uncertainty_train += uncertainty.kinetic_input_uncertainties[i]
    else:
        print(src_dict.keys())
print(n_rate_rules, uncertainty_rate_rules)
print(n_pdep, uncertainty_pdep)
print(n_lib, uncertainty_lib)
print(n_training, uncertainty_train)


In [ ]:
def nn(pct):
    return int(np.round(pct * (n_rate_rules + n_pdep + n_lib + n_training) / 100.0))

%matplotlib inline
plt.figure(figsize=(8.0, 4), dpi=80)
plt.clf()
plt.pie([n_rate_rules, n_pdep, n_lib, n_training], startangle=90, autopct=nn)
ax = plt.gca()
ax.legend(['Rate Rules', 'Pressure Dependent Network', 'Library', 'Training'], loc='right', bbox_to_anchor=(1.8, 0.8))
plt.title('Reaction Sources')
plt.savefig('reaction_sources.png')
plt.show()

In [ ]:
%matplotlib inline
plt.clf()
plt.pie([uncertainty_rate_rules, uncertainty_pdep, uncertainty_lib, uncertainty_train], startangle=90)
ax = plt.gca()
ax.legend(['Rate Rules', 'Pressure Dependent Network', 'Library', 'Training'], loc='right', bbox_to_anchor=(1.8, 0.8))
plt.title('Model Uncertainty of Reactions')
plt.show()

In [ ]:
%matplotlib inline
plt.clf()
plt.pie([n_rate_rules, n_pdep, n_lib, n_training], startangle=90, labels=[n_rate_rules, n_pdep, n_lib, n_training])
ax = plt.gca()
ax.legend(['Rate Rules', 'Pressure Dependent Network', 'Library', 'Training'], loc='right', bbox_to_anchor=(1.8, 0.8))
plt.title('Reaction Sources')
plt.show()